In [2]:
import os
os.environ['MKL_THREADING_LAYER'] = 'GNU'

from collections import Counter

import keras
import nltk
import numpy as np

from keras import Sequential, layers



/home/aleksey/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
from nltk.corpus import gutenberg, brown, reuters

# for fileid in gutenberg.fileids():
#     num_chars = len(gutenberg.raw(fileid))
#     num_words = len(gutenberg.words(fileid))
#     num_sents = len(gutenberg.sents(fileid))
#     num_vocab = len(set(w.lower() for w in gutenberg.words(fileid)))
#     print(round(num_chars/num_words), round(num_words/num_sents), round(num_words/num_vocab), fileid)

In [5]:
fileids = None
# fileids = brown.fileids()[:250]
# sents_brown = list(brown.sents(fileids=fileids))

fileids = None
fileids = ['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt']
sents_gutenberg = list(gutenberg.sents(fileids=fileids))

sents = sents_gutenberg

print('Sentences:', len(sents))

Sentences: 16498


In [6]:
words = set(nltk.flatten(sents))

print('Words:', len(words))

Words: 11482


In [30]:
capital = []

for s in sents:
    for i, w in enumerate(s, start=1):
        if i != 1 and w.istitle():
            capital.append(w)

capital = np.array(capital)


In [34]:
caps = Counter(capital)

In [38]:
caps

Counter({'Fulton': 15,
         'County': 83,
         'Grand': 31,
         'Jury': 4,
         'Friday': 66,
         'City': 160,
         'Executive': 9,
         'Committee': 88,
         'Atlanta': 35,
         'September-October': 1,
         'Superior': 16,
         'Court': 131,
         'Judge': 57,
         'Durwood': 1,
         'Pye': 1,
         'Ivan': 4,
         'Allen': 20,
         'Jr.': 75,
         'Only': 80,
         'Purchasing': 2,
         'Department': 162,
         'Legislature': 27,
         'State': 285,
         'Welfare': 7,
         'This': 628,
         'Failure': 1,
         'Bar': 14,
         'Association': 87,
         'These': 296,
         'Jan.': 20,
         'Jail': 4,
         'Police': 18,
         'Tax': 9,
         'Office': 48,
         'Bellwood': 1,
         'Alpharetta': 1,
         'Grady': 5,
         'Hospital': 34,
         'Health': 21,
         'William': 189,
         'B.': 75,
         'Hartsfield': 4,
         'Pearl': 7,
    

In [8]:
data = ''.join(words)
SEQ_LENGTH = 100

chars = list(set(data))

VOCAB_SIZE = len(chars)

SEQS_NUM = int(np.floor(len(data)/SEQ_LENGTH))

ix_to_char = {ix:char for ix, char in enumerate(chars)}
char_to_ix = {char:ix for ix, char in enumerate(chars)}

print('SEQ_LENGTH =', SEQ_LENGTH, '\nNumber of sequences:', SEQS_NUM, '\nVOCAB_SIZE =', VOCAB_SIZE)

SEQ_LENGTH = 100 
Number of sequences: 849 
VOCAB_SIZE = 80


In [9]:
X = np.zeros((SEQS_NUM, SEQ_LENGTH, VOCAB_SIZE))
y = np.zeros((SEQS_NUM, SEQ_LENGTH, VOCAB_SIZE))

for i in range(0, SEQS_NUM):
    last_index = i * SEQ_LENGTH
    
    X_sequence = data[last_index : last_index + SEQ_LENGTH]

    X_sequence_ix = [char_to_ix[value] for value in X_sequence]
    
    input_sequence = np.zeros((SEQ_LENGTH, VOCAB_SIZE))
    
    for j in range(SEQ_LENGTH):
        a = X_sequence_ix[j]
            
        input_sequence[j][a] = 1.
        
    X[i] = input_sequence

    y_sequence = data[last_index + 1 : last_index + SEQ_LENGTH + 1]
    y_sequence_ix = [char_to_ix[value] for value in y_sequence]
    
    target_sequence = np.zeros((SEQ_LENGTH, VOCAB_SIZE))
    
    for j in range(SEQ_LENGTH):
        target_sequence[j][y_sequence_ix[j]] = 1.
    
    y[i] = target_sequence

In [10]:
HIDDEN_DIM = 500
LAYER_NUM = 3

model = Sequential()

model.add(layers.LSTM(HIDDEN_DIM, input_shape=(None, VOCAB_SIZE), return_sequences=True))

for i in range(LAYER_NUM - 1):
    model.add(layers.LSTM(HIDDEN_DIM, return_sequences=True))

model.add(layers.TimeDistributed(layers.Dense(VOCAB_SIZE)))
model.add(layers.Activation('softmax'))

model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

In [11]:
def generate_text(model, length):
    ix = [np.random.randint(VOCAB_SIZE)]
    
    y_char = [ix_to_char[ix[-1]]]
    
    X = np.zeros((1, length, VOCAB_SIZE))
    
    for i in range(length):
        X[0, i, :][ix[-1]] = 1
    
        print(ix_to_char[ix[-1]], end="")
        
        ix = np.argmax(model.predict(X[:, :i+1, :])[0], 1)
        
        y_char.append(ix_to_char[ix[-1]])
    
    return ('').join(y_char)

In [12]:
GENERATE_LENGTH = 100
generate_text(model, GENERATE_LENGTH)

0!![[[[yyyy--JJJeeeeeeedddddAAAA..._______WWW]]]]BBBBB''''zppp??taaaaMMMMggggggggDDDnnnnn888888888SS

"0!![[[[yyyy--JJJeeeeeeedddddAAAA..._______WWW]]]]BBBBB''''zppp??taaaaMMMMggggggggDDDnnnnn888888888SSS"

In [13]:
nb_epoch = 0
BATCH_SIZE = 50

while nb_epoch < 100:
    print('\n\n')
    model.fit(X, y, batch_size=BATCH_SIZE, verbose=1, nb_epoch=1)
    nb_epoch += 1
    generate_text(model, GENERATE_LENGTH)
    if nb_epoch % 2 == 0:
        model.save_weights('trained/checkpoint_{}_epoch_{}.hdf5'.format(HIDDEN_DIM, nb_epoch))

/home/aleksey/anaconda3/lib/python3.6/site-packages/keras/models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/1
350/849 [===========>..................] - ETA: 1:21 - loss: 3.7531

KeyboardInterrupt: 